<a href="https://drive.google.com/file/d/1K2nPJlLel_Kuay-xexGpsyXjLNaChJut/view?usp=sharing" target="_blank" >
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>

In [0]:
# Import necessary libraries
import os
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import Image
from tensorflow.keras import backend as K
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Input, Conv2D, MaxPooling2D, InputLayer, ReLU

%matplotlib inline



In [0]:
# Initialize a sequential model
model = ___

# Add a convolution layer with 32 filters with kernel of size 3, relu activation, same padding
# Remember to find out the dimension of the input and send it as input_shape
___

# Add a convolution layer with 32 filters with kernel of size 3, relu activation, same padding
___

# Add a maxpooling layer with pool size of 2
___

# Add a convolution layer with 64 filters with kernel of size 3, relu activation, same padding
___

# Add a convolution layer with 64 filters with kernel of size 3, relu activation, same padding
___

# Add a maxpooling layer with pool size of 2
___

# Add a convolution layer with 128 filters with kernel of size 3, relu activation, same padding
___

# Add a convolution layer with 128 filters with kernel of size 3, relu activation, same padding
___

# Add a maxpooling layer with pool size of 2
___

# Add a Flatten layer
___

# Add a fully connected dense layer with 1024 nodes and relu activation
___

# Add a fully connected dense layer with 1024 nodes and relu activation
___

# Add a dense layer for the output with 10 nodes and softmax activation
___

# Compile the model with SGD optimizer with learning rate as 0.001 and momentum of 0.9
# Use categorical cross-entropy as the loss and accuracy as the metric
___


In [0]:
# Take a look at the summary of the model
___

In [0]:
# As training this model takes a long time, we have trained it for you 
# The pre-trained model weights are present in the file "receptive_field_weights.h5"
# Load these weights to skip the .fit step
___

# GETTING THE ACTIVATIONS

In [0]:
# Heler function to load the images given the file path
def load_image(img_path):

    img = image.load_img(img_path, target_size=(32, 32))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor = preprocess_input(img_tensor)               # imshow expects values in the range [0, 1]

    return img_tensor

In [0]:
# Path of the images folder
path = "cifar/"

# Get a list of all files within the path
dirs = os.listdir(path)

# Select a convolutional layer by specifying its number
layer_num = ___

# Select the activation map within the selected layer to see 
# its corresponding receptive field
activation_num = ___


In [0]:
# Get the output of the selected activation map using tensorflow backend
inp = model.input
outputs = [layer.output for layer in model.layers]
functors = [K.function([inp],[out]) for out in outputs]


In [0]:
# Initialize a list to store the output of the activation map for each image
acts = []

# Run a loop for the number of input images i.e.50
for i in range(1,51):
    
  # Define the image path of each image in the cifar folder
  file_path = path+"img"+str(i)+".jpg"

  # Call the load_image function defined for every image
  # above to get the images
  img_path = ___

  # Get the model layer output for the specific layer using functors got above
  layer_outs = ___

  # Get the model output for the specific layer based on layer_num
  last = ___

  # Get the set of feature maps of the selected layer
  feature_map = last[0]

  # Get the required feature maps out of the feature map set
  select_act = feature_map[0][:,:,activation_num]

  # Save the selected activation to a list
  acts.append(select_act.flatten())

In [0]:
# Find out which images activates which pixel of the feature map the most

# Initialize a dictionary to hold the pixel-wise image that activates it the most
best_img = {}

# Loop over the entire pixel space of the selected activation
# The dimension of the feature map can be got from model.layers[layer_num].output_shape
for j in range(___):
  # Define a list to store the activation of each image for a given pixel of the feature map  
  l = []
  for i in range(len(acts)):
    l.append(acts[i][j])
    
  # Get the image that activates a particular pixel of the activation map the most
  best_img[j] = np.argmax(l)+1



# SETUP THE PADDING, KERNEL AND STRIDES

In [0]:
# To get the receptive field we have computed the padding size, kernel size 
# and stride of each layer for the defined cnn model above

# Get the model padding, kernel and stride list
pad = [1,1,0,1,1,0,1,1,0]
ker = [3,3,2,3,3,2,3,3,2]
strd = [1,1,2,1,1,2,1,1,2]

# Helper function to get a dictionary that is of the following form
# [layer name]:{kernel, stride, padding}
def make_arch(model):
  c = 0
  arc_dict = {}
  for i in model.layers[:9]:
    l = []
    l.append(ker[c])
    l.append(strd[c])
    l.append(pad[c])
    c+=1
    arc_dict[i.name] = l
  return arc_dict

In [0]:
# Call the above defined make_arch function to get the dictionary required
arc_dict = make_arch(model)

In [0]:
# Helper class to compute the receptive field size and start

# To define a receptive field, we need the size of the receptive field
# and the position of the starting pixel

class ComputeReceptiveField():
    def calculate(self, architecture, input_image_size):
        arc = []
        input_layer = ('input_layer', input_image_size, 1, 1, 0.5)
        #self._print_layer_info(input_layer)
        arc.append(input_layer)
        for key in architecture:
            current_layer = self._calculate_layer_info(architecture[key], input_layer, key)
            input_layer = current_layer
            arc.append(current_layer)
        return arc
            
            
    def _calculate_layer_info(self, current_layer, input_layer, layer_name):
        n_in = input_layer[1]
        j_in = input_layer[2]
        r_in = input_layer[3]
        start_in = input_layer[4]
        
        k = current_layer[0]
        s = current_layer[1]
        p = current_layer[2]

        n_out = math.floor((n_in - k + 2*p)/s) + 1
        padding = (n_out-1)*s - n_in + k 
        p_right = math.ceil(padding/2)
        p_left = math.floor(padding/2)
        

        j_out = j_in * s
        r_out = r_in + (k - 1)*j_in
        start_out = start_in + ((k-1)/2 - p_left)*j_in
        return layer_name, n_out, j_out, r_out, start_out

In [0]:
# Create an object of the receptive field class
rf = ComputeReceptiveField()

# Call the calculate method of the receptive field class by passing the dictionary 
# created above and the input image size i.e. 32

# This method returns 4 values: 
# layer name, n[output size],j[cummulative stride] ,r[receptive field size], s[start]
com = rf.calculate(arc_dict, 32)


In [0]:
# Helper function to get the receptive field of the input image i.e. patch seen 
# by each pixel of the feature map

# The function takes the image, pixel position along x and y axis
# and the layer number
def get_rf(img, y_pix=0, x_pix=0, ln=7):

  # Get the layer_number  
  layer_num = ln

  # Get the start point i.e. the center pixel of the receptive field
  start = float(np.array(com[layer_num])[4]) + x_pix

  # Get the size of the receptive field
  rf = int(np.array(com[layer_num])[3])

  # Get the leftmost pixel of the receptive patch
  lt = start - math.ceil(rf/2) 

  # Get the rightmost pixel of the receptive patch
  rt = start + math.ceil(rf/2) 

  # Get the topmost pixel of the receptive patch
  top = start - math.ceil(rf/2) + y_pix

  # Get the bottom-most pixel of the receptive patch
  btm = start + math.ceil(rf/2) + y_pix

  # Clip the left pixel if it lower than 0
  if lt<0:
    lt=0

  # Clip the right pixel if it is higher than the max size of the input
  if rt>img[0].shape[0]:
    rt=img[0].shape[0]-1

  # Clip the top if it is lower than 0
  if top<0:
    top=0

  # Clip the bottom if it is higher than the max size of the input 
  if btm>img[0].shape[1]:
    btm=img[0].shape[1]-1

  # Get pixels corresponding to the receptive field from the input image
  img_rf = img[0][int(lt):int(rt), int(top):int(btm), :]

  # Return the receptive field of the pixel
  return img_rf


In [0]:
# Define a dictionary to store the receptive field associated to each pixel
rf_dict = {}

# Run a counter to use as dictionary keys
counter=0

# Loop over the possible pixel values of the feature map to get the receptive fiels
for i in range(model.layers[layer_num].output_shape[1]):
  for j in range(model.layers[layer_num].output_shape[2]):

    # Call the load_image function that is defined 
    # using the the best_img dictionary for each pixel
    img = load_image("cifar/img"+str(best_img[counter])+".jpg")

    # Call the get_rf function to get the receptive field for 
    # that pixel and the most activating image i.e img
    img_rf = get_rf(img, i, j, layer_num)

    # Save the returned receptive field as the values of the dictionary
    rf_dict[counter] = img_rf

    # Increment the counter
    counter+=1


In [0]:
# Helper code to plot the receptive fields associated with each pixel

print("Layer number: "+str(layer_num))
print("Receptive field size: ", int(np.array(com[layer_num])[3]))

fig, ax = plt.subplots(model.layers[layer_num].output_shape[1], model.layers[layer_num].output_shape[2], figsize = (10,10))
counter = 0

# Loop over the possible pixel values of the feature map to plot the receptive fiels
for i in range(model.layers[layer_num].output_shape[1]):
  for j in range(model.layers[layer_num].output_shape[2]):

    # Denormalize the pixel values
    ax[i][j].imshow((rf_dict[counter]/2)+0.5)
    ax[i][j].axis('off')
    counter+=1
 
plt.tight_layout()
